In [1]:
import requests
import json
import numpy as np
import pandas as pd

In [2]:
from config import gkey, fb_key

In [3]:
csv_path = "Project1_AmazonSites.csv"
city_csv = pd.read_csv(csv_path)
# to remove all null values
city_df = city_csv.iloc[:39,:9]
# change zip to only integer
city_df['zip code'] = city_df['zip code'].astype(int)
for index, zip in city_df['zip code'].iteritems():
    if len(str(zip)) == 4:
        city_df['zip code'].iloc[index] = "0"+str(zip)
city_df.head()

C:\Users\hleao\AppData\Local\Continuum\anaconda3\envs\Python\lib\site-packages\pandas\core\indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


,amazon city,site,site name,address,city,state,zip code,latitude,longitude
0,Northern Virginia Area,Site1,Center for Innovation Technology,2214 Rock Hill Road,Herndon,VA,20170,38.96,-77.42
1,Northern Virginia Area,Site2,Crystal City and Potomac Yard,3535 S Ball Street,Arlington,VA,22202,38.84,-77.05
2,Washington DC,Site1,Nationals Park/ D.C United Stadium,32-60 R Street,Washington,DC,20024,38.86,-77.01
3,Washington DC,Site2,Union Station,10 G Street,Washington,DC,20002,38.89,-77.00
4,Washington DC,Site3,Hill-East Neighborhood,1430 Pennsylvania Avenue,Washington,DC,20003,38.88,-76.97


In [4]:
city_param = city_df[['latitude', 'longitude', 'address','site name','city','amazon city']]
city_param

,latitude,longitude,address,site name,city,amazon city
0,38.9600,-77.4200,2214 Rock Hill Road,Center for Innovation Technology,Herndon,Northern Virginia Area
1,38.8400,-77.0500,3535 S Ball Street,Crystal City and Potomac Yard,Arlington,Northern Virginia Area
2,38.8600,-77.0100,32-60 R Street,Nationals Park/ D.C United Stadium,Washington,Washington DC
3,38.8900,-77.0000,10 G Street,Union Station,Washington,Washington DC
4,38.8800,-76.9700,1430 Pennsylvania Avenue,Hill-East Neighborhood,Washington,Washington DC
5,38.9100,-77.0200,1701 8th Street,Shaw-Howard University Area,Washington,Washington DC
6,34.1802,-118.6028,21801 Oxnard Street,Warner Center West San Fernando Valley,Woodland Hills,Los Angeles
7,34.4318,-118.5584,24155 Newhall Ranch Road,New Hall Ranch Santa Clarita Valley,Santa Clarita,Los Angeles
8,34.0885,-117.7702,1101 McKinley Avenue,Pomona Fairplex,Pomona,Los Angeles
9,42.3900,-71.0000,525 McChellan Highway,Suffolk Downs,Boston,Boston


In [5]:
airport_list = []

## Airports
for city in city_param.values:
    target_coordinates = f"{city[0]}, {city[1]}"
    #radius of within 31 miles or 50000 meters
    target_radius = 50000
    target_type = "airport"
    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # set up a parameters dictionary
    params = {
        "location": target_coordinates,
        "radius": target_radius,
        "type": target_type,
        "key": gkey
    }

    # run a request using our params dictionary
    response = requests.get(base_url, params=params)
    airport_json = response.json()
    for airport_num in range(len(airport_json['results'])):
        airport_str = airport_json['results'][airport_num]['name']
        if 'rating' in airport_json['results'][airport_num].keys():
            airport_rating = airport_json['results'][airport_num]['rating']
        else:
            airport_rating = None
        airport_list.append({'Site Name':city[3],
                     'City':city[4],
                     'Amazon City':city[5],
                     'Airport':airport_str,
                     'Airport Rating':airport_rating})
airport_df = pd.DataFrame(airport_list)
len(set(airport_df['Airport']))

235

In [6]:
#clean up any NaN
airport_df.replace(["NaN", 'NaT'], np.nan, inplace = True)
airport_df = airport_df.dropna()
print(airport_df.head())
len(set(airport_df['Airport']))

                                     Airport  Airport Rating  \
1               Dulles International Airport             3.9   
2  Ronald Reagan Washington National Airport             3.9   
3                  Manassas Regional Airport             4.1   
4                 Leesburg Executive Airport             4.2   
5                 College Park Airport - CGS             4.9   

              Amazon City     City                         Site Name  
1  Northern Virginia Area  Herndon  Center for Innovation Technology  
2  Northern Virginia Area  Herndon  Center for Innovation Technology  
3  Northern Virginia Area  Herndon  Center for Innovation Technology  
4  Northern Virginia Area  Herndon  Center for Innovation Technology  
5  Northern Virginia Area  Herndon  Center for Innovation Technology  


141

In [7]:
amazon_city_group = airport_df.groupby(['Amazon City','City','Site Name','Airport'])
final_amazon_city_df = amazon_city_group.mean()


In [8]:
final_amazon_city_df

Airport Rating
Amazon City   City       Site Name                   Airport                                                           
Atlanta       Atlanta    Downtown Atlanta            Atlanta Regional Airport - Falcon Field                        4.4
                                                     Atlantic Aviation PDK                                          4.2
                                                     Cobb County International Airport                              4.4
                                                     Fulton County Airport-Brown Field                              4.8
                                                     Gwinnett County Airport (LZU)                                  4.4
                                                     Hartsfield Jackson Atlanta International Airpor...             4.0
                                                     Hartsfield-Jackson Atlanta International Airport               3.9
                                                     Lola Landing                                                   4.5
                                                     Matthews                                                       5.0
                                                     Morgan Falls Heliport                                          1.0
                                                     Peachtree Dekalb Airport                                       4.3
                         Former GM plant             Atlantic Aviation PDK                                          4.2
                                                     Cherokee County Airport                                        5.0
                                                     Cobb County International Airport                              4.4
                                                     Fulton County Airport-Brown Field                              4.8
                                                     Georgia Limousine                                              5.0
                                                     Gwinnett County Airport (LZU)                                  4.4
                                                     Hartsfield Jackson Atlanta International Airpor...             4.0
                                                     Hartsfield-Jackson Atlanta International Airport               3.9
                                                     Lola Landing                                                   4.5
                                                     Matthews                                                       5.0
                                                     Morgan Falls Heliport                                          1.0
                                                     Peachtree Dekalb Airport                                       4.3
                         Midtown                     Malvern Municipal Airport                                      4.0
                                                     Memorial Field Airport                                         4.3
                         The High Street Site        Atlantic Aviation PDK                                          4.2
                                                     Cartersville Airport                                           5.0
                                                     Cherokee County Airport                                        5.0
                                                     Cobb County International Airport                              4.4
                                                     Fulton County Airport-Brown Field                              4.8
...                                                                                                                 ...
Washington DC Washington Shaw-Howard University Area A2 Global Shipping                                             5.0
                         

In [9]:
#Filter rows without aiport in the name
final_amazon_city_df = final_amazon_city_df.reset_index()

In [10]:
# final_amazon_city_df = final_amazon_city_df.drop(any,axis = 1)
# final_amazon_city_df

In [11]:
airport_avg_rating_by_site_df = final_amazon_city_df[final_amazon_city_df["Airport"].str.contains("Airport") == True]
airport_avg_rating_by_site_df = airport_avg_rating_by_site_df[airport_avg_rating_by_site_df["Airport"].str.contains("Office") == False]
airport_avg_rating_by_site_df = airport_avg_rating_by_site_df[airport_avg_rating_by_site_df["Airport"].str.contains("Support") == False]
airport_avg_rating_by_site_df = airport_avg_rating_by_site_df[airport_avg_rating_by_site_df["Airport"].str.contains("Parking") == False]
airport_avg_rating_by_site_df = airport_avg_rating_by_site_df[airport_avg_rating_by_site_df["Airport"].str.contains(".com") == False]

airport_avg_rating_by_site_df.to_csv('site_airport_avg_rating.csv')

In [12]:
airport_avg_rating_by_site_df

,Amazon City,City,Site Name,Airport,Airport Rating
0,Atlanta,Atlanta,Downtown Atlanta,Atlanta Regional Airport - Falcon Field,4.4
2,Atlanta,Atlanta,Downtown Atlanta,Cobb County International Airport,4.4
3,Atlanta,Atlanta,Downtown Atlanta,Fulton County Airport-Brown Field,4.8
4,Atlanta,Atlanta,Downtown Atlanta,Gwinnett County Airport (LZU),4.4
6,Atlanta,Atlanta,Downtown Atlanta,Hartsfield-Jackson Atlanta International Airport,3.9
10,Atlanta,Atlanta,Downtown Atlanta,Peachtree Dekalb Airport,4.3
12,Atlanta,Atlanta,Former GM plant,Cherokee County Airport,5.0
13,Atlanta,Atlanta,Former GM plant,Cobb County International Airport,4.4
14,Atlanta,Atlanta,Former GM plant,Fulton County Airport-Brown Field,4.8
16,Atlanta,Atlanta,Former GM plant,Gwinnett County Airport (LZU),4.4


In [29]:
#rating average per amazon city
# Need to dedupe the airport per city since multiple sites are within 31 miles of each other
#DataFrame with only airport, amaozon city, and airport rating
condensed_airport_df = airport_avg_rating_by_site_df[['Amazon City','Airport','Airport Rating']]
condensed_airport_df = condensed_airport_df.drop_duplicates()
condensed_airport_df
#df.drop_duplicates(subset=['A', 'C'], keep=False)

,Amazon City,Airport,Airport Rating
0,Atlanta,Atlanta Regional Airport - Falcon Field,4.4
2,Atlanta,Cobb County International Airport,4.4
3,Atlanta,Fulton County Airport-Brown Field,4.8
4,Atlanta,Gwinnett County Airport (LZU),4.4
6,Atlanta,Hartsfield-Jackson Atlanta International Airport,3.9
10,Atlanta,Peachtree Dekalb Airport,4.3
12,Atlanta,Cherokee County Airport,5.0
23,Atlanta,Malvern Municipal Airport,4.0
24,Atlanta,Memorial Field Airport,4.3
26,Atlanta,Cartersville Airport,5.0


In [30]:
airport_by_city_df = condensed_airport_df.groupby(['Amazon City'])
avg_airport_rating_by_city_df = airport_by_city_df.mean()
avg_airport_rating_by_city_df

,Airport Rating
Amazon City,
Atlanta,4.450000
Austin,4.277778
Boston,3.700000
Chicago,4.390000
Los Angeles,4.295000
New York,3.980000
Northern Virginia Area,4.040000
Raleigh,4.200000
Washington DC,4.244444


In [31]:
airport_count_by_city = airport_by_city_df['Amazon City'].count()
airport_count_by_city_df = pd.DataFrame(airport_count_by_city)
airport_count_by_city_df = airport_count_by_city_df.rename(columns={"Amazon City": "Count"})
airport_count_by_city_df = airport_count_by_city_df.reset_index()

In [32]:
airport_count_by_city_df.join(avg_airport_rating_by_city_df, on = "Amazon City")

,Amazon City,Count,Airport Rating
0,Atlanta,10,4.450000
1,Austin,9,4.277778
2,Boston,5,3.700000
3,Chicago,10,4.390000
4,Los Angeles,20,4.295000
5,New York,10,3.980000
6,Northern Virginia Area,10,4.040000
7,Raleigh,6,4.200000
8,Washington DC,9,4.244444


In [17]:
condensed_airport_df[condensed_airport_df['Amazon City'].values == "Los Angeles"]

,Amazon City,Airport,Airport Rating
109,Los Angeles,Bob Hope Airport,4.4
110,Los Angeles,Los Angeles International Airport,3.5
111,Los Angeles,Van Nuys Airport,4.4
112,Los Angeles,Los Angeles County Mens Detention Center - Mai...,5.0
113,Los Angeles,Camarillo Airport,4.9
114,Los Angeles,Hawthorne Municipal Airport,4.3
116,Los Angeles,Agua Dulce Airpark,5.0
117,Los Angeles,Clay Lacy Aviation,3.2
118,Los Angeles,Signature Flight Support VNY West - Van Nuys A...,5.0
119,Los Angeles,AAKASH TRAVEL,4.0


In [35]:
# splitting municpal airports, internationl airports, and other
for index, airport in enumerate(condensed_airport_df['Airport'].values):
    condensed_airport_df['Is International'] = 0

for index, airport in enumerate(condensed_airport_df['Airport'].values):
    if "International" in str(airport):
        condensed_airport_df['Is International'].iloc[index] = 1
    else:
        condensed_airport_df['Is International'].iloc[index] = 0

condensed_airport_df

C:\Users\hleao\AppData\Local\Continuum\anaconda3\envs\Python\lib\site-packages\pandas\core\indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


,Amazon City,Airport,Airport Rating,Is International
0,Atlanta,Atlanta Regional Airport - Falcon Field,4.4,0
2,Atlanta,Cobb County International Airport,4.4,1
3,Atlanta,Fulton County Airport-Brown Field,4.8,0
4,Atlanta,Gwinnett County Airport (LZU),4.4,0
6,Atlanta,Hartsfield-Jackson Atlanta International Airport,3.9,1
10,Atlanta,Peachtree Dekalb Airport,4.3,0
12,Atlanta,Cherokee County Airport,5.0,0
23,Atlanta,Malvern Municipal Airport,4.0,0
24,Atlanta,Memorial Field Airport,4.3,0
26,Atlanta,Cartersville Airport,5.0,0


In [52]:
con_airport_groupby = condensed_airport_df.groupby('Amazon City')
cond_airport_group_df = pd.DataFrame(con_airport_groupby['Amazon City'].count())
cond_airport_group_df = cond_airport_group_df.rename( columns={"Amazon City": "Airport Count"})
cond_airport_group_df['International Airports'] = con_airport_groupby['Is International'].sum()
cond_airport_group_df = cond_airport_group_df.rename( columns={"Is International": "International Airport Count"})
cond_airport_group_df = cond_airport_group_df.sort_values(by=['Airport Count', 'International Airports'], ascending=False)
cond_airport_group_df
order_list = [1,2,3,3,3,4,5,6,7]
cond_airport_group_df['Rank'] = order_list
print(cond_airport_group_df)
cond_airport_group_df.to_csv('Airport ranking.csv')

                        Airport Count  International Airports  Rank
Amazon City                                                        
Los Angeles                        20                       3     1
Chicago                            10                       3     2
Atlanta                            10                       2     3
New York                           10                       2     3
Northern Virginia Area             10                       2     3
Washington DC                       9                       2     4
Austin                              9                       1     5
Raleigh                             6                       1     6
Boston                              5                       1     7


Classifying the airports https://gist.github.com/branis/5109902

In [ ]:
json_path = 'airports.json'
airport_json = pd.read_json(json_path)
airport_json = airport_json.rename(index=str, columns={"name": "Airport"})


In [ ]:
airportcheck = list(set(airport_df['Airport']))
airport_list = airport_json['Airport']

for airportcheckrow in airportcheck:
    for airport in airport_list.values:
        if airport == airportcheckrow:
            print(f"{airportcheckrow} is in the airport_json")

# try:
#     final_amazon_city_df.join(airport_json, on='Airport', how='left', lsuffix='_left')
# except:
#     print('no airport')
# final_amazon_city_df.head()

Json above is uselsss

In [ ]:
json_path2 = 'airport2.json'
airport2_json = pd.read_json(json_path2)
airport2_json.head()

In [ ]:
# airportcheck = list(set(airport_df['Airport']))
# airport2_list = airport2_json['Airport']

# for airportcheck in airportcheck:
#     for airport in airport2_list.values:
#         if airport == airportcheck:
#             print(f"{airportcheck} is in the airport2_json")

In [ ]:
#from https://github.com/jbrooksuk/JSON-Airports/blob/master/airports.json
json_path3 = 'airport3.json'
airport3_json = pd.read_json(json_path3)
airport3_json = airport3_json.rename(index=str, columns={"name": "Airport"})
airport3_json.head()

In [ ]:
airportcheck = list(set(airport_df['Airport']))
airport3_list = airport3_json['Airport']

for airportcheckrow in airportcheck:
    for airport in airport3_list.values:
        if  airportcheckrow == airport:
            print(f"{airportcheckrow} is in the airport3_json")

In [ ]:
#Facebook API on airports
#https://developers.facebook.com/docs/places/web/search

In [ ]:
# test out FB api

fb_base_url = "https://graph.facebook.com/search"
fb_params = {
            "type":"place",
            "center":"38.96,-77.42",
            "distance":50000,
            "q":"airport",
            "fields":"name",
            "access_token":fb_key
             }
response = requests.get(fb_base_url, params=fb_params)
fb_airport_json = response.json()

fb_airport_json

# Can't extract any facebook data because of the cambridge analytica.        